In [1]:
import os
import pathlib
import logging
import re
import shutil
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tarfile
import time
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
import numpy as np

%load_ext tensorboard
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [2]:
import tensorflow.compat.v1 as tf1
import contextlib2
import tensorflow as tf
import wget

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util
# from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
# from object_detection.utils import colab_utils
from object_detection.builders import model_builder
from object_detection.dataset_tools import tf_record_creation_util


# Enable GPU dynamic memory allocation
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
#LINK : http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
model_name = "efficientdet_d0_coco17_tpu-32"
model = "efficientdet_d0_coco17_tpu-32.tar.gz"

# os.makedirs("pre_trained_models/", exist_ok=True)
download_tar = f"http://download.tensorflow.org/models/object_detection/tf2/20200711/{model}"
wget.download(download_tar, out="pre_trained_models/")

tar = tarfile.open(f"pre_trained_models/{model}")
tar.extractall(path="pre_trained_models/")
tar.close()

os.unlink(f"pre_trained_models/{model}")

In [5]:
import re

pipeline_path = "pre_trained_models/pipeline_olahan_daging.config"
fine_tune_checkpoint = "pre_trained_models/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0"
train_input_path = "dataset/olahan_daging/train/food.tfrecord"
valid_input_path = "dataset/olahan_daging/valid/food.tfrecord"
label_map_path = "dataset/olahan_daging/train/food_label_map.pbtxt"
num_classes = 6
train_batch_size = 8
min_dim = 256
max_dim = 256

with open(pipeline_path) as f:
    pipeline_content = f.read()
with open(pipeline_path, 'w') as f:
    # change to finetune checkpoint path
    pipeline_content = re.sub('fine_tune_checkpoint: ".*?"',
               f'fine_tune_checkpoint: "{fine_tune_checkpoint}"', pipeline_content)

    # change to tfrecord path for train and validation
    pipeline_content = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', f'input_path: "{train_input_path}"', pipeline_content)
    pipeline_content = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', f'input_path: "{valid_input_path}"', pipeline_content)

    # change to label_map_path
    pipeline_content = re.sub(
        'label_map_path: ".*?"', f'label_map_path: "{label_map_path}"', pipeline_content)

    # Set training batch_size
    pipeline_content = re.sub(r'(train_config\s*{\s*batch_size:\s*)\d+', r'\g<1>{}'.format(train_batch_size), pipeline_content)

    # Set the dimension
    pipeline_content = re.sub('min_dimension: [0-9]+',
               f'min_dimension: {min_dim}', pipeline_content)
    pipeline_content = re.sub('max_dimension: [0-9]+',
               f'max_dimension: {max_dim}', pipeline_content)

    # Set number of classes num_classes.
    pipeline_content = re.sub('num_classes: [0-9]+',
               f'num_classes: {num_classes}', pipeline_content)

    # change the finetune type to detection
    pipeline_content = re.sub(
        'fine_tune_checkpoint_type: "classification"', f'fine_tune_checkpoint_type: "{"detection"}"', pipeline_content)

    f.write(pipeline_content)

#run on terminal

!python models/research/object_detection/model_main_tf2.py --pipeline_config_path=pre_trained_models/pipeline_olahan_daging.config --num_train_steps=20000 --model_dir=training/olahan_daging_efficientdet_d0 --alsologtostderr

#run on terminal with above so mAP score shows

!python models/research/object_detection/model_main_tf2.py --pipeline_config_path=pre_trained_models/pipeline_olahan_daging.config --model_dir=training/olahan_daging_efficientdet_d0 --checkpoint_dir=training/olahan_daging_efficientdet_d0 --alsologtostderr --eval_timeout=1000

tensorboard --logdir=training/olahan_daging_efficientdet_d0

#run on terminal

!python models/research/object_detection/exporter_main_v2.py --trained_checkpoint_dir=training/olahan_daging_efficientdet_d0 --output_directory=custom_model_lite/olahan_daging_efficientdet_d0 --pipeline_config_path=pre_trained_models/pipeline_olahan_daging.config


In [27]:
import re

# Sample data string
data = """
item {
  name: "ayam bakar"
  id: 1
  display_name: "ayam bakar"
}
item {
  name: "ayam goreng"
  id: 2
  display_name: "ayam goreng"
}
item {
  name: "ikan goreng"
  id: 3
  display_name: "ikan goreng"
}
item {
  name: "sate"
  id: 4
  display_name: "sate"
}
item {
  name: "steak"
  id: 5
  display_name: "steak"
}
"""

# Define the regular expression patterns for name, id, and display_name
# name_pattern = r'name:\s*"([^"]*)"'
id_pattern = r'id:\s*(\d+)'
display_name_pattern = r'display_name:\s*"([^"]*)"'

# Find all matches using regex
# names = re.findall(name_pattern, data)
ids = [int(i) for i in re.findall(id_pattern, data)]
display_names = re.findall(display_name_pattern, data)

# Structure the data as required
result = {}
for i in range(len(display_names)):
    result[display_names[i]] = {ids[i]: {'id': ids[i], 'name': display_names[i]}}

print(result)


{'ayam bakar': {1: {'id': 1, 'name': 'ayam bakar'}}, 'ayam goreng': {2: {'id': 2, 'name': 'ayam goreng'}}, 'ikan goreng': {3: {'id': 3, 'name': 'ikan goreng'}}, 'sate': {4: {'id': 4, 'name': 'sate'}}, 'steak': {5: {'id': 5, 'name': 'steak'}}}


In [25]:
names

['ayam bakar',
 'ayam bakar',
 'ayam goreng',
 'ayam goreng',
 'ikan goreng',
 'ikan goreng',
 'sate',
 'sate',
 'steak',
 'steak']

In [37]:
model_cat = [p.split("_efficientdet_d0")[0] for p in os.listdir("custom_model_lite")]
# models = {}
category_indexs = {}

def read_label_file(label_path):
    id_pattern = r'id:\s*(\d+)'
    display_name_pattern = r'display_name:\s*"([^"]*)"'
    with open(label_path, 'r') as file:
        pbtxt_content = file.read()
        ids = [int(i) for i in re.findall(id_pattern,  pbtxt_content)]
        display_names = re.findall(display_name_pattern, pbtxt_content)
    result = {}
    for i in range(len(display_names)):
        result[ids[i]] = {'id': ids[i], 'name': display_names[i]}
    return result

for cat in model_cat:
    # models[cat] = tf.saved_model.load(f"custom_model_lite\{cat}_efficientdet_d0\saved_model")
    category_indexs[cat] = read_label_file(f"dataset/{cat}/train/food_label_map.pbtxt")

In [38]:
category_indexs

{'daging': {1: {'id': 1, 'name': 'ayam bakar'},
  2: {'id': 2, 'name': 'ayam goreng'},
  3: {'id': 3, 'name': 'ikan goreng'},
  4: {'id': 4, 'name': 'sate'},
  5: {'id': 5, 'name': 'steak'}},
 'jajanan': {1: {'id': 1, 'name': 'bakwan'},
  2: {'id': 2, 'name': 'batagor'},
  3: {'id': 3, 'name': 'bika_ambon'},
  4: {'id': 4, 'name': 'martabak telur'},
  5: {'id': 5, 'name': 'pempek'}},
 'karbo': {1: {'id': 1, 'name': 'bihun'},
  2: {'id': 2, 'name': 'mie'},
  3: {'id': 3, 'name': 'nasi goreng'},
  4: {'id': 4, 'name': 'nasi putih'},
  5: {'id': 5, 'name': 'roti'}},
 'lauk': {1: {'id': 1, 'name': 'kerupuk'},
  2: {'id': 2, 'name': 'tahu'},
  3: {'id': 3, 'name': 'telur'},
  4: {'id': 4, 'name': 'tempe'}},
 'sayur': {1: {'id': 1, 'name': 'capcay'},
  2: {'id': 2, 'name': 'gado-gado'},
  3: {'id': 3, 'name': 'soto'},
  4: {'id': 4, 'name': 'terong balado'},
  5: {'id': 5, 'name': 'tumis kangkung'}}}

In [23]:
category_indexs

{'daging': {1: {'id': 1, 'name': 'ayam bakar'},
  2: {'id': 2, 'name': 'ayam goreng'},
  3: {'id': 3, 'name': 'ikan goreng'},
  4: {'id': 4, 'name': 'sate'},
  5: {'id': 5, 'name': 'steak'}},
 'jajanan': {1: {'id': 1, 'name': 'bakwan'},
  2: {'id': 2, 'name': 'batagor'},
  3: {'id': 3, 'name': 'bika_ambon'},
  4: {'id': 4, 'name': 'martabak telur'},
  5: {'id': 5, 'name': 'pempek'}},
 'karbo': {1: {'id': 1, 'name': 'bihun'},
  2: {'id': 2, 'name': 'mie'},
  3: {'id': 3, 'name': 'nasi goreng'},
  4: {'id': 4, 'name': 'nasi putih'},
  5: {'id': 5, 'name': 'roti'}},
 'lauk': {1: {'id': 1, 'name': 'kerupuk'},
  2: {'id': 2, 'name': 'tahu'},
  3: {'id': 3, 'name': 'telur'},
  4: {'id': 4, 'name': 'tempe'}},
 'sayur': {1: {'id': 1, 'name': 'capcay'},
  2: {'id': 2, 'name': 'gado-gado'},
  3: {'id': 3, 'name': 'soto'},
  4: {'id': 4, 'name': 'terong balado'},
  5: {'id': 5, 'name': 'tumis kangkung'}}}

In [10]:
obj_detected = []
IMAGE_PATHS = "testing/nasi_ikan.jpg"

image_np = np.array(Image.open(IMAGE_PATHS))
input_tensor = tf.convert_to_tensor(image_np, dtype=tf.uint8)
input_tensor = input_tensor[tf.newaxis, ...]
input_tensor = input_tensor[:, :, :, :3]

In [11]:
for cat in model_cat:
    detections = models[cat]((input_tensor))
    classes = detections['detection_classes'][0].numpy()
    scores = detections['detection_scores'][0].numpy()
    for i in range(len(scores)):
        if((scores[i] > 0.5) and (scores[i] <= 1.0)):
            object_name = category_indexs[cat][classes[i]]
            obj_detected.append(object_name)

In [17]:
category_indexs

{'daging': {1: {'id': 1, 'name': 'ayam bakar'},
  2: {'id': 2, 'name': 'ayam goreng'},
  3: {'id': 3, 'name': 'ikan goreng'},
  4: {'id': 4, 'name': 'sate'},
  5: {'id': 5, 'name': 'steak'}},
 'jajanan': {1: {'id': 1, 'name': 'bakwan'},
  2: {'id': 2, 'name': 'batagor'},
  3: {'id': 3, 'name': 'bika_ambon'},
  4: {'id': 4, 'name': 'martabak telur'},
  5: {'id': 5, 'name': 'pempek'}},
 'karbo': {1: {'id': 1, 'name': 'bihun'},
  2: {'id': 2, 'name': 'mie'},
  3: {'id': 3, 'name': 'nasi goreng'},
  4: {'id': 4, 'name': 'nasi putih'},
  5: {'id': 5, 'name': 'roti'}},
 'lauk': {1: {'id': 1, 'name': 'kerupuk'},
  2: {'id': 2, 'name': 'tahu'},
  3: {'id': 3, 'name': 'telur'},
  4: {'id': 4, 'name': 'tempe'}},
 'sayur': {1: {'id': 1, 'name': 'capcay'},
  2: {'id': 2, 'name': 'gado-gado'},
  3: {'id': 3, 'name': 'soto'},
  4: {'id': 4, 'name': 'terong balado'},
  5: {'id': 5, 'name': 'tumis kangkung'}}}

In [12]:
obj_detected

['ikan goreng', 'nasi putih']

In [13]:
with open('dataset/olahan_daging/train/food_label_map.pbtxt', 'r') as file:
    pbtxt_content = file.read()

In [16]:
pattern = r'display_name:\s*"([^"]*)"'
display_names = re.findall(pattern, pbtxt_content)
display_names

['bakso', 'nugget', 'opor ayam', 'rendang', 'sosis', 'udang']

In [20]:
obj_detected = []
IMAGE_PATHS = "nasi_ikan.jpg"

for p in os.listdir("custom_model_lite"):
    cat = p.split("_efficientdet_d0")[0]
    saved_model_loaded = tf.saved_model.load(f"custom_model_lite\{p}\saved_model")
    category_index = label_map_util.create_category_index_from_labelmap(f"{cat}/train/food_label_map.pbtxt", use_display_name=True)
    image_np = np.array(Image.open(IMAGE_PATHS))
    input_tensor = tf.convert_to_tensor(image_np, dtype=tf.uint8)
    input_tensor = input_tensor[tf.newaxis, ...]
    input_tensor = input_tensor[:, :, :, :3]
    detections = saved_model_loaded(input_tensor)
    classes = detections['detection_classes'][0].numpy()
    scores = detections['detection_scores'][0].numpy()
    for i in range(len(scores)):
        if((scores[i] > 0.5) and (scores[i] <= 1.0)):
            object_name = category_index[classes[i]]['name']
            obj_detected.append(object_name)
    

In [21]:
obj_detected

['ikan goreng', 'nasi putih']

In [11]:
obj_detected = []

for d in detections_array:
    classes = d['detection_classes'][0].numpy()
    scores = d['detection_scores'][0].numpy()

4

In [40]:
saved_model_loaded = tf.saved_model.load("custom_model_lite\jajanan_efficientdet_d0\saved_model")
PATH_TO_LABELS = 'jajanan/train/food_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

IMAGE_PATHS = "photo.jpg"
image_np = np.array(Image.open(IMAGE_PATHS))
input_tensor = tf.convert_to_tensor(image_np, dtype=tf.uint8)
input_tensor = input_tensor[tf.newaxis, ...]
input_tensor = input_tensor[:, :, :, :3]

In [41]:
category_index[1]['name']

'bakwan'

In [42]:
image_np = np.array(Image.open(IMAGE_PATHS))
input_tensor = tf.convert_to_tensor(image_np, dtype=tf.uint8)
input_tensor = input_tensor[tf.newaxis, ...]
input_tensor = input_tensor[:, :, :, :3]
input_tensor.shape

TensorShape([1, 400, 400, 3])

In [43]:
detections = saved_model_loaded(input_tensor)

In [14]:
classes = detections['detection_classes'][0].numpy()
scores = detections['detection_scores'][0].numpy()

In [15]:
classes

array([5., 3., 2., 5., 3., 1., 5., 5., 2., 1., 1., 2., 5., 3., 5., 2., 5.,
       5., 2., 1., 2., 2., 5., 3., 5., 2., 2., 3., 3., 1., 5., 5., 5., 2.,
       2., 2., 5., 1., 5., 1., 5., 5., 5., 2., 3., 2., 5., 5., 1., 5., 5.,
       1., 1., 5., 1., 3., 4., 5., 2., 4., 5., 1., 2., 3., 2., 2., 1., 1.,
       5., 3., 4., 1., 5., 5., 2., 1., 5., 2., 3., 4., 2., 1., 5., 5., 2.,
       5., 3., 5., 1., 5., 1., 3., 3., 2., 4., 3., 2., 5., 1., 1.],
      dtype=float32)

In [46]:
scores

array([[0.01394221, 0.01222091, 0.01155228, 0.01058487, 0.01026587,
        0.01023106, 0.00995212, 0.00923526, 0.00905502, 0.00875791,
        0.00850109, 0.00847665, 0.00838382, 0.00838023, 0.00818047,
        0.00780999, 0.00756392, 0.00756196, 0.00752028, 0.00749922,
        0.0074649 , 0.00736557, 0.00733423, 0.00731031, 0.00721338,
        0.00703817, 0.00686638, 0.00679421, 0.00676177, 0.00675764,
        0.00656289, 0.00653557, 0.00644133, 0.00639698, 0.00638087,
        0.00636783, 0.0062957 , 0.00629158, 0.00626436, 0.00622538,
        0.00619953, 0.00603963, 0.00602442, 0.00600811, 0.00597259,
        0.00594399, 0.00590838, 0.00587545, 0.00586731, 0.00585866,
        0.00583623, 0.0057706 , 0.00576108, 0.00576054, 0.00573587,
        0.0057319 , 0.00570947, 0.00570738, 0.0056954 , 0.00567021,
        0.0056359 , 0.00563022, 0.0056287 , 0.00562349, 0.00559751,
        0.00559558, 0.00551532, 0.00550622, 0.0054363 , 0.00543624,
        0.00540517, 0.0053587 , 0.00533089, 0.00

In [47]:
len(classes[0])

100

In [48]:
len(scores[0])

100

In [49]:
obj_detected = []
for i in range(len(scores[0])):
    if((scores[0][i] > 0.5) and (scores[0][i] <= 1.0)):
        object_name = category_index[classes[0][i]]['name']
        obj_detected.append(object_name)

In [50]:
obj_detected

[]